In [257]:
import os
import sys
import copy
from typing import List, Tuple, Dict
from sage.graphs.connectivity import connected_components

Util

In [258]:
def generate_transposition(n: int, u: int, v: int) -> Permutation:
    action = list(range(1, n + 1))
    action[u - 1] = v
    action[v - 1] = u
    return Permutation(action)

generate_transposition(4, 2, 4)  # [1, 4, 3, 2]

[1, 4, 3, 2]

In [259]:
indent = 0


def iprint(s: str, pre: int = 0, post: int = 0):
    global indent
    indent += pre
    print("    " * indent + s)
    indent += post

Input verification

In [260]:
def validate_edges(n: int, allowed_pairs: Tuple[Tuple[int, int], ...]):
    """Checks for invalid edges, duplicate edges (up to ordering), and self-loops."""
    elements = [u for pair in allowed_pairs for u in pair]
    if not (1 <= min(elements) and max(elements) <= n):
        print("Element out of range")
        return False

    for (u, v) in allowed_pairs:
        if u == v:
            print(f"Self-loop found: {(u, v)}")
            return False

    pairs = set([(min(u, v), max(u, v)) for (u, v) in allowed_pairs])
    if len(pairs) != len(allowed_pairs):
        iprint("Duplicate edge found.")
        return False

    if not Graph([list(range(1, n + 1)), list(allowed_pairs)]).is_connected():
        print("Graph is not connected")
        return False

    return True

CC = connected component

`dp[rank of permutation, CC's] = (min length, number of factorizations with such length)`

In [261]:
Edge = Tuple[int, int]

def merge_cc(cc_list, u: int, v: int):
    cc_u = [cc for cc in cc_list if u in cc]
    cc_v = [cc for cc in cc_list if v in cc]
    assert len(cc_u) == 1 and len(cc_v) == 1
    cc_u = cc_u[0]
    cc_v = cc_v[0]
    
    if cc_u == cc_v:
        return cc_list
    new_cc_list = list(cc_list)
    new_cc_list.remove(cc_u)
    new_cc_list.remove(cc_v)
    new_cc = tuple(sorted(cc_u + cc_v))
    new_cc_list.append(new_cc)
    new_cc_list.sort()
    return tuple(map(tuple, new_cc_list))


def compute_dp(n: int,
               allowed_edges: List[Tuple[int, int]],
               dp,
               rank: int,
               cc_list) -> Tuple[int, int]:
    if (rank, cc_list) in dp:
        return dp[(rank, cc_list)]

    if rank == 0 and len(cc_list) == 1:
        dp[(rank, cc_list)] = (0, 1)
        return dp[(rank, cc_list)]

    iprint(f"compute_dp: {Permutations(n).unrank(rank)}, cc_list={cc_list}", 0, 1)
    dp[(rank, cc_list)] = None
    perm = Permutations(n).unrank(rank)

    cur_min_length = -1
    cur_count = 0

    # Try all transpositions
    for (u, v) in allowed_edges:
        # Get new rank
        action = generate_transposition(n, u, v)
        new_perm = Permutation(perm).left_action_product(action)
        new_rank = Permutations(n).rank(new_perm)

        # Get new cc list
        new_cc_list = merge_cc(cc_list, u, v)

        # Update DP
        next_dp = compute_dp(n, allowed_edges, dp, new_rank, new_cc_list)
        if next_dp is None:
            continue
        iprint(f"next_dp: {Permutations(n).unrank(new_rank)}, new_cc_list={new_cc_list}")
        if cur_min_length == -1 or next_dp[0] < cur_min_length:
            cur_min_length, cur_count = next_dp
        elif next_dp[0] == cur_min_length:
            cur_count += next_dp[1]
        iprint(f"{cur_min_length, cur_count}", 1, -1)

    dp[(rank, cc_list)] = (cur_min_length + 1, cur_count)
    iprint(f"end: {dp[(rank, cc_list)]}", -1, 0)
    return dp[(rank, cc_list)]

`compute_length_n(n)`: Returns a map from permutation rank to (min length of factorization, number of such factorizations). Keys are ranks of permutations of length n.

In [262]:
def compute_length_n(n: int, allowed_edges: List[Tuple[int, int]]):
    global indent
    indent = 0
    starting_cc_tuple = tuple((i,) for i in range(1, n + 1))

    dp = {}
    perm_n = Permutations(n)
    for rank in range(factorial(n)):
        perm = perm_n.unrank(rank)
        print(f"--- start rank={rank}, perm={perm} ---")
        compute_dp(n, allowed_edges, dp, rank, starting_cc_tuple)
        min_length, count = dp[(rank, starting_cc_tuple)]
        print(f"{rank}: {perm}: min_len = {min_length}, count = {count}")
    result = {rank: dp[(rank, starting_cc_tuple)] for rank in range(factorial(n))}
    return result

In [263]:
def main(n: int, allowed_edges: Tuple[Tuple[int, int], ...]):
    if not validate_edges(n, allowed_edges):
        return
    results = compute_length_n(n, allowed_edges)

    perm_n = Permutations(n)
    for rank, res in results.items():
        print(f"{perm_n.unrank(rank)}: {res}")
    return results

In [264]:
n = 3
test_allowed_pairs = ((1, 2), (1, 3))
main(n, test_allowed_pairs)

--- start rank=0, perm=[1, 2, 3] ---
compute_dp: [1, 2, 3], cc_list=((1,), (2,), (3,))
    compute_dp: [2, 1, 3], cc_list=((1, 2), (3,))
        compute_dp: [1, 2, 3], cc_list=((1, 2), (3,))
            compute_dp: [3, 2, 1], cc_list=((1, 2, 3),)
                compute_dp: [2, 3, 1], cc_list=((1, 2, 3),)
                    compute_dp: [1, 3, 2], cc_list=((1, 2, 3),)
                        compute_dp: [3, 1, 2], cc_list=((1, 2, 3),)
                            compute_dp: [2, 1, 3], cc_list=((1, 2, 3),)
                                next_dp: [1, 2, 3], new_cc_list=((1, 2, 3),)
                                    (0, 1)
                            end: (1, 1)
                            next_dp: [2, 1, 3], new_cc_list=((1, 2, 3),)
                                (1, 1)
                        end: (2, 1)
                        next_dp: [3, 1, 2], new_cc_list=((1, 2, 3),)
                            (2, 1)
                    end: (3, 1)
                    next_dp: [1, 3, 2], new_c

{0: (4, 3), 1: (3, 2), 2: (3, 2), 3: (2, 1), 4: (2, 1), 5: (3, 1)}